<img src="images/evaluation_engine.png">

In [25]:
import pandas as pd
import sklearn.metrics
from sklearn.metrics import mean_squared_error
import math
import os
from os import listdir
from os.path import isfile, join

In [26]:
def mse_len_tuple(y_train, y_pred):
    return (mean_squared_error(y_pred=y_pred, y_true=y_train), len(y_pred))

In [27]:
def make_mse_entry(train_file, pred_file, mse_df):
    train = pd.read_csv(train_file, sep=",", na_values=["?",",","#","NaN","unknown",""])
    pred = pd.read_csv(pred_file, sep=",", na_values=["?",",","#","NaN","unknown",""])
    mse, nrows = mse_len_tuple(train["sales"], pred["sales"])
    mse_df.loc[-1] = [mse, nrows]  # adding a row
    mse_df.index = mse_df.index + 1  # shifting index
    mse_df = mse_df.sort_index()  # sorting by index


In [28]:
def get_mse_df(train_files, pred_files):
    mse_df = pd.DataFrame(columns = ["mse", "nrows"])
    for i in range(0, len(pred_files)):
        make_mse_entry(train_files[i], pred_files[i], mse_df)
    #print(mse_df.head())
    return mse_df

In [29]:
def rmse(mse_df):
    mse_df_copy = mse_df.copy()
    mse_df_copy["nmse"] = mse_df_copy["mse"] * mse_df_copy["nrows"]
    total_nmse = mse_df_copy.sum(axis = 0, skipna = True)["nmse"]
    total_nrows = mse_df_copy.sum(axis = 0, skipna = True)["nrows"]
    rmse_score = math.sqrt(total_nmse/total_nrows)
    return rmse_score

In [30]:
def evaluate(train_files, pred_files):
    mse_df = get_mse_df(train_files, pred_files)
    return rmse(mse_df)

In [31]:
def get_files(files_dir, files_prefix):
    return [join(files_dir, f) for f in listdir(files_dir) if isfile(join(files_dir, f)) and files_prefix in f]

In [32]:
train_files = get_files("/Users/ggupta14581/Desktop/PHD/Models/Prophet/prophet_1/", "simple_train_1")
pred_files = get_files("/Users/ggupta14581/Desktop/PHD/Models/Prophet/prophet_1/", "pred_1")
evaluate(train_files, pred_files)

86.01480931742846